In [1]:
import gym
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

np.random.seed(17)
tf.random.set_seed(17)

In [2]:
env = gym.make("CartPole-v0")  # создадим среду
env.seed(17)  # зафиксируем сид для воспроизводимости

[17]

In [3]:
num_inputs = 4  # кол-во параметров состояния среды
num_actions = 2  # кол-во возможных действий

num_hidden = 64

# создадим нейросеть при помощи Functional API
# т.к. нам потребуется 2 выхода

inputs = layers.Input(shape=(num_inputs,))
common = layers.Dense(num_hidden, activation="relu")(inputs)
common_1 = layers.Dense(num_hidden, activation="relu")(common)
action = layers.Dense(num_actions, activation="softmax")(common_1)
critic = layers.Dense(1)(common_1)

model = keras.Model(inputs=inputs, outputs=[action, critic])

In [4]:
optimizer = keras.optimizers.Adam(learning_rate=0.01)
loss = keras.losses.Huber()

In [5]:
# зададим discount-фактор для награды
gamma = 0.995

# как долго длится эпизод
max_steps_per_episode = 10000

# минимальная машинная точность
eps = np.finfo(np.float32).eps.item()

# выделим списки для хранения значений
# вероятности действий, критика и награды
action_probs_history = []
critic_value_history = []
rewards_history = []


running_reward = 0
episode_count = 0

while True:
    # сбрасываем состояние к изначальному и записываем его
    state = env.reset()
    episode_reward = 0
    
    # проводим раунд игры, записывая работу нашей нейросети в GradientTape
    with tf.GradientTape() as tape:
        # идём по раунду шагами
        for timestep in range(1, max_steps_per_episode):
            env.render()  # для графического отображения окна с игрой

            state = tf.convert_to_tensor(state)
            state = tf.expand_dims(state, 0)

            # зная состояние, модель должна нам предсказать действие 
            # и поправку от критика
            action_probs, critic_value = model(state)
            
            # поправку критика сразу сохраним
            critic_value_history.append(critic_value[0, 0])

            # выберем действие, исходя из предсказанных вероятностей
            action = np.random.choice(num_actions, p=np.squeeze(action_probs))
            
            # вероятность выбранного действия сохраняем в историю
            # можно сразу и прологарифмировать, как того требует наш лосс
            action_probs_history.append(tf.math.log(action_probs[0, action]))

            # говорим среде выполнить выбранное действие
            # и получаем от неё следующее состояние и награду
            # если игра окончена, среда вернут done=True
            state, reward, done, _ = env.step(action)
            
            # записываем полученную награду
            rewards_history.append(reward)
            
            episode_reward += reward

            if done:
                break

        # пересчитываем running_reward с затуханием
        running_reward = 0.05 * episode_reward + (1 - 0.05) * running_reward

        # награда на каждом шаге игры (то, что нам и должен предсказать критик)
        # определяется как суммарная награда от этого и всех последующих шагов (с затуханием)
        # считаем их все в 1 проход и добавляем в returns
        returns = []
        discounted_sum = 0
        for r in rewards_history[::-1]:
            discounted_sum = r + gamma * discounted_sum
            returns.insert(0, discounted_sum)

        # нормализуем их для повышения стабильности обучения
        returns = np.array(returns)
        returns = (returns - np.mean(returns)) / (np.std(returns) + eps)
        returns = returns.tolist()

        # осталось посчитать значения функций потерь, чтобы затем осуществить шаг градиентного спуска
        history = zip(action_probs_history, critic_value_history, returns)
        actor_losses = []
        critic_losses = []
        for log_prob, value, ret in history:
            # на каждом шаге считаем лосс для actor и critic
            # у первого это просто награда (за вычетом поправки от критика), 
            # умноженная на логарифмическую вероятность действия
            diff = ret - value
            actor_losses.append(-log_prob * diff)  # actor loss

            # критик же оканчивается обычной регрессией:
            # он должен уметь предсказывать ожидаемую награду на каждом шаге
            critic_losses.append(
                loss(tf.expand_dims(value, 0), tf.expand_dims(ret, 0))
            )

        # теперь можем осуществить стандартный шаг градиентного спуска
        # чтобы запустить его из одной точки, сложим все лоссы
        # т.к. при суммировании градиент от каждого слагаемого отправится
        # только в направлении данного слагаемого, это то, что нужно
        loss_value = sum(actor_losses) + sum(critic_losses)
        grads = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        # перед следующим эпизодом очистим все списки
        action_probs_history.clear()
        critic_value_history.clear()
        rewards_history.clear()

    episode_count += 1
    
    # каждые 10 эпизодов будем выводить running reward
    if episode_count % 10 == 0:
        print(f"Episode: {episode_count}, running reward: {running_reward:.2f}")

    # если running reward стал больше 195, цель достигнута
    if running_reward > 195:
        print(f"Solved at episode {episode_count}!")
        break

# саму среду в конце нужно закрыть
# чтобы рендер прекратился и можно было дальше работать в питоне
env.close()

Episode: 10, running reward: 8.15
Episode: 20, running reward: 11.60
Episode: 30, running reward: 12.10
Episode: 40, running reward: 11.82
Episode: 50, running reward: 12.53
Episode: 60, running reward: 13.42
Episode: 70, running reward: 12.62
Episode: 80, running reward: 16.84
Episode: 90, running reward: 22.16
Episode: 100, running reward: 28.86
Episode: 110, running reward: 34.70
Episode: 120, running reward: 29.82
Episode: 130, running reward: 26.64
Episode: 140, running reward: 28.68
Episode: 150, running reward: 31.56
Episode: 160, running reward: 40.36
Episode: 170, running reward: 57.56
Episode: 180, running reward: 84.54
Episode: 190, running reward: 90.89
Episode: 200, running reward: 92.80
Episode: 210, running reward: 132.84
Episode: 220, running reward: 123.89
Episode: 230, running reward: 115.69
Episode: 240, running reward: 135.04
Episode: 250, running reward: 156.74
Episode: 260, running reward: 167.78
Episode: 270, running reward: 163.31
Episode: 280, running reward: 1

In [6]:
state = env.reset()
state = tf.convert_to_tensor(state)
state = tf.expand_dims(state, 0)
action_probs, _ = model(state)

In [7]:
action_probs

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.8301848 , 0.16981524]], dtype=float32)>